Constructing LeanDojo Benchmark (Lean 4)
===================================

This script uses [LeanDojo](https://leandojo.org/) to construct LeanDojo Benchmark 4 in the appendix of our paper:

[LeanDojo: Theorem Proving with Retrieval-Augmented Language Models](https://leandojo.org/)      
Under review at NeurIPS (Datasets and Benchmarks Track), 2023  
[Kaiyu Yang](https://yangky11.github.io/), [Aidan Swope](https://aidanswope.com/about), [Alex Gu](https://minimario.github.io/), [Rahul Chalamala](https://rchalamala.github.io/), [Peiyang Song](https://peiyang-song.github.io/), [Shixing Yu](https://billysx.github.io/), [Saad Godil](https://www.linkedin.com/in/saad-godil-9728353/), [Ryan Prenger](https://www.linkedin.com/in/ryan-prenger-18797ba1/), [Anima Anandkumar](http://tensorlab.cms.caltech.edu/users/anima/)

The dataset is constructed from [mathlib4](https://github.com/leanprover-community/mathlib4/tree/355541ae7a2455222f179dcf7f074aa2c45eb8aa) (`355541ae7a2455222f179dcf7f074aa2c45eb8aa`) and will be saved to `../leandojo_benchmark_4`. It includes 2000 theorems for validation, 2000 theorems for testing, and the rest for training. Please refer to our paper for details. For most use cases, you shouldn't need to generate the data and can directly use our official LeanDojo Benchmark 4 downloadable [here](https://zenodo.org/record/8040110).

This script is for Lean 4. We also have a [version for Lean 3](https://github.com/lean-dojo/LeanDojo/blob/main/scripts/generate-benchmark-lean3.ipynb).


In [1]:
import ray
import json
import shutil
import random
import networkx as nx
from tqdm import tqdm
from copy import copy
from pathlib import Path
from loguru import logger
from datetime import datetime
from collections import defaultdict
from ray.util.actor_pool import ActorPool
from typing import Dict, List, Tuple, Union

import lean_dojo
from lean_dojo import *
from lean_dojo.constants import LEAN4_DEPS_DIR

random.seed(3407)  # https://arxiv.org/abs/2109.08203

URL = "https://github.com/leanprover-community/mathlib4"
COMMIT = "355541ae7a2455222f179dcf7f074aa2c45eb8aa"
DST_DIR = Path("../leandojo_benchmark_4")
NUM_VAL = NUM_TEST = 2000

## Splitting the Theorems

We will split the theorems into train/val/test using two different strategies.

In [2]:
SPLIT_NAME = str  # train/val/test
SPLIT = Dict[SPLIT_NAME, List[TracedTheorem]]
SPLIT_STRATEGY = str

### Splitting Randomly

The first and the simplest strategy is splitting the theorems randomly, which can be implemented by a random shuffle followed by a sequential split.

In [3]:
def _split_sequentially(
    traced_theorems: List[TracedTheorem],
) -> SPLIT:
    """Split ``traced_theorems`` sequentially into train/val/test."""
    num_theorems = len(traced_theorems)
    num_train = num_theorems - NUM_VAL - NUM_TEST
    return {
        "train": traced_theorems[:num_train],
        "val": traced_theorems[num_train : num_train + NUM_VAL],
        "test": traced_theorems[num_train + NUM_VAL :],
    }


def split_randomly(
    traced_theorems: List[TracedTheorem],
) -> SPLIT:
    """Split ``traced_theorems`` randomly into train/val/test."""
    logger.info("Splitting the theorems randomly")
    traced_theorems = copy(traced_theorems)
    random.shuffle(traced_theorems)
    return _split_sequentially(traced_theorems)

### Splitting by Premise

The second strategy is splitting by premise. We want to test the prover's capability in using novel premises, i.e., premises that have never been used in training. Please see the implementation below. Note that validation and testing theorems may share premises. So the **testing performance should be reported using models trained on the training set only, NOT training plus validation.**

In [4]:
def split_by_premise(
    traced_theorems: List[TracedTheorem],
) -> SPLIT:
    """
    Split theorems into train/val/test so that proofs in val/test rely on at
    least one novel premise that does not appear in train.
    """
    logger.info("Splitting the theorems by premises")

    # Figure out the number of theorems in train/val/test.
    num_theorems = len(traced_theorems)
    num_val_test = NUM_VAL + NUM_TEST
    num_train = num_theorems - num_val_test
    theorems_val_test = set()

    # Map each premise to a list of theorems using it.
    theorems_by_premises = defaultdict(list)
    for t in traced_theorems:
        for p in t.get_premise_full_names():
            theorems_by_premises[p].append(t)

    # Sort the premises by the number of theorems using them (in ascending order).
    theorems_by_premises = sorted(theorems_by_premises.items(), key=lambda x: len(x[1]))

    # For each premise, put all theorems using it into val_test so that it does not appear in train.
    for _, thms in theorems_by_premises:
        if len(theorems_val_test) < num_val_test:
            theorems_val_test.update(thms)

    # All other theorems go to train.
    theorems_train = [t for t in traced_theorems if t not in theorems_val_test]
    theorems_val_test = list(theorems_val_test)
    random.shuffle(theorems_val_test)

    return {
        "train": theorems_train,
        "val": theorems_val_test[:NUM_VAL],
        "test": theorems_val_test[NUM_VAL:],
    }

Given a traced repo, we can split the theorems using these strategies.

In [5]:
def split_data(traced_repo: TracedRepo) -> Dict[SPLIT_STRATEGY, SPLIT]:
    # Skip theorems in the Lean 4 repo itself.
    traced_theorems = [
        thm for thm in traced_repo.get_traced_theorems() if not thm.repo.is_lean4
    ]
    logger.info(f"{len(traced_theorems)} theorems in total")

    return {
        "random": split_randomly(traced_theorems),
        "novel_premises": split_by_premise(traced_theorems),
    }

## Exporting the Data
Once theorems are splitted into train/val/test. We export them to JSON formats that can be easily used in machine learning.

In [6]:
def _get_file_path(traced_repo: TracedRepo, thm: TracedTheorem) -> str:
    if thm.repo == traced_repo.repo:
        # The theorem belongs to the traced repo itself.
        return str(thm.theorem.file_path)
    else:
        # The theorem belongs to one of the dependencies.
        for name, dep in traced_repo.dependencies.items():
            if dep == thm.repo:
                return f"{LEAN4_DEPS_DIR}/{name}/{thm.theorem.file_path}"
        raise ValueError(f"Unable to find the dependency {thm.repo}")


def export_proofs(
    traced_repo: TracedRepo, splits: Dict[SPLIT_STRATEGY, SPLIT], dst_path: Path
) -> None:
    """Export all proofs in a traced repo to ``dst_path''."""
    for strategy, split in splits.items():
        split_dir = dst_path / strategy
        split_dir.mkdir(parents=True)

        for name, theorems in split.items():
            data = []
            num_tactics = 0

            for thm in theorems:
                tactics = [
                    {
                        "tactic": t.tactic,
                        "annotated_tactic": t.get_annotated_tactic(),
                        "state_before": t.state_before,
                        "state_after": t.state_after,
                    }
                    for t in thm.get_traced_tactics()
                    if t.state_before != "no goals"
                    and "·" not in t.tactic  # Ignore "·".
                ]
                num_tactics += len(tactics)
                data.append(
                    {
                        "url": thm.repo.url,
                        "commit": thm.repo.commit,
                        "file_path": _get_file_path(traced_repo, thm),
                        "full_name": thm.theorem.full_name,
                        "start": list(thm.start),
                        "end": list(thm.end),
                        "traced_tactics": tactics,
                    }
                )
            oup_path = split_dir / f"{name}.json"
            json.dump(data, oup_path.open("wt"))
            logger.info(
                f"{len(theorems)} theorems and {num_tactics} tactics saved to {oup_path}"
            )


def export_premises(traced_repo: TracedRepo, dst_path: Path) -> None:
    """Export all premise definitions in a traced repo to ``dst_path``."""
    oup_path = dst_path / "corpus.jsonl"
    num_premises = 0

    with oup_path.open("wt") as oup:
        G = traced_repo.traced_files_graph

        for tf_node in reversed(list(nx.topological_sort(G))):
            tf = G.nodes[tf_node]["traced_file"]
            imports = [str(_) for _ in G.successors(tf_node)]
            premises = tf.get_premise_definitions()
            num_premises += len(premises)
            oup.write(
                json.dumps(
                    {"path": str(tf.path), "imports": imports, "premises": premises}
                )
                + "\n"
            )
    logger.info(
        f"{num_premises} theorems/definitions from {traced_repo.num_traced_files} files saved to {oup_path}"
    )


def export_licenses(traced_repo: TracedRepo, dst_path: Path) -> None:
    """Export the licenses of a traced repo and all its dependencies to ``dst_path``."""
    license_dir = dst_path / "licenses"
    license_dir.mkdir()
    all_repos = [traced_repo.repo] + list(traced_repo.dependencies.values())

    for repo in all_repos:
        lic = repo.get_license()
        if lic is None:
            continue
        with (license_dir / repo.name).open("wt") as oup:
            oup.write(lic)

    with (license_dir / "README.md").open("wt") as oup:
        oup.write(
            "This directory contains licenses of Lean repos used to generate this dataset. The dataset itself is released under [CC BY 2.0](https://creativecommons.org/licenses/by/2.0/)."
        )


def export_metadata(traced_repo: TracedRepo, dst_path: Path, **kwargs) -> None:
    """Export the metadata of a traced repo to ``dst_path''."""
    metadata = dict(kwargs)
    metadata["creation_time"] = str(datetime.now())
    metadata["from_repo"] = {
        "url": traced_repo.repo.url,
        "commit": traced_repo.repo.commit,
    }
    metadata["leandojo_version"] = lean_dojo.__version__
    json.dump(metadata, (dst_path / "metadata.json").open("wt"))


def export_data(
    traced_repo: TracedRepo,
    splits: Dict[SPLIT_STRATEGY, SPLIT],
    dst_path: Union[str, Path],
    **kwargs,
) -> None:
    """Export a traced repo whose theorems have been splitted to ``dst_path``."""
    if isinstance(dst_path, str):
        dst_path = Path(dst_path)
    if dst_path.exists():
        logger.warning(f"{dst_path} already exists. Removing it now.")
        shutil.rmtree(dst_path)

    # Export the proofs.
    export_proofs(traced_repo, splits, dst_path)

    # Export the premises (theorems, definitions, etc.).
    export_premises(traced_repo, dst_path)

    # Export the licenses.
    export_licenses(traced_repo, dst_path)

    # Export metadata.
    export_metadata(traced_repo, dst_path, **kwargs)

In [7]:
repo = LeanGitRepo(URL, COMMIT)
traced_repo = trace(repo)
splits = split_data(traced_repo)
export_data(traced_repo, splits, DST_DIR, dataset_name="LeanDojo Benchmark 4")

2023-08-11 09:54:33.708 | INFO     | lean_dojo.data_extraction.trace:trace:163 - Loading the traced repo from /home/kaiyu/.cache/lean_dojo/leanprover-community-mathlib4-355541ae7a2455222f179dcf7f074aa2c45eb8aa/mathlib4
2023-08-11 09:54:35,869	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|████████████████████████████████████████| 4264/4264 [08:13<00:00,  8.64it/s]
2023-08-11 10:03:54.861 | INFO     | __main__:split_data:4 - 100780 theorems in total
2023-08-11 10:03:54.863 | INFO     | __main__:split_randomly:18 - Splitting the theorems randomly
2023-08-11 10:03:54.903 | INFO     | __main__:split_by_premise:8 - Splitting the theorems by premises
2023-08-11 10:06:41.243 | INFO     | __main__:export_proofs:49 - 96780 theorems and 200914 tactics saved to ../leandojo_benchmark_4/random/train.json
2023-08-11 10:06:44.887 | INFO     | __main__:export_proofs:49 - 2000 theorems and 4041 tactics saved to ../leandojo_benchmark_4/random/val.json
202

## Data Format

This is the resulting data directory:

```
├─corpus.jsonl
├─metadata.json
├─licenses
│ ├─lean4
│ ├─mathlib4
│ ├─doc-gen4
│ ├─aesop
│ ├─ProofWidgets4
│ ├─std4
│ └─README.md
├─random
│ ├─train.json
│ ├─val.json
│ └─test.json
└─novel_premises
  ├─train.json
  ├─val.json
  └─test.json
```

`corpus.jsonl` is a corpus of all theorems and definitions in mathlib4 that can potentially be used as premises. Sub-directories `random` and `novel_premise` are different strategies for splitting the theorems. For each strategy, we have `*.json` files for train/val/test. The sub-directory `licenses` contains license information.

### Corpus of Potential Premises

`corpus.jsonl` is in [JSON Lines format](https://jsonlines.org/); a line includes the potential premises defined in a single `*.lean` file.

In [8]:
!cat ../leandojo_benchmark_4/corpus.jsonl | wc -l

4264


Let's look at one of them.

In [13]:
corpus_path = DST_DIR / "corpus.jsonl"
lines = list(corpus_path.open())
file_in_corpus = json.loads(lines[2000])
file_in_corpus.keys()

dict_keys(['path', 'imports', 'premises'])

We can check the file's path and other files it imports.

In [14]:
file_in_corpus["path"], file_in_corpus["imports"]

('Mathlib/Analysis/SpecialFunctions/Complex/Arg.lean',
 ['Mathlib/Analysis/SpecialFunctions/Trigonometric/Inverse.lean',
  'lake-packages/lean4/src/lean/Init.lean',
  'Mathlib/Analysis/SpecialFunctions/Trigonometric/Angle.lean'])

In [15]:
len(file_in_corpus["premises"])

71

We can inspect the first potential premise:

In [16]:
file_in_corpus["premises"][0]

{'full_name': 'Complex.sin_arg',
 'code': 'theorem sin_arg (x : ℂ) : Real.sin (arg x) = x.im / abs x',
 'start': [36, 1],
 'end': [40, 60],
 'kind': 'commandtheoremn'}

Each premise has a fully qualified name, its definition (in the form of Lean code), and the exact location it is defined.


### Theorems/Proofs Data

Now let's take a look at the theorems/proofs data, taking the `random` split as an example.

In [17]:
train_path = DST_DIR / "random/train.json"
proofs_train = json.load(train_path.open())
len(proofs_train)

96780

Each element in `proofs_val` represents a theorem. Let's check one of them.

In [19]:
for proof in proofs_train:
    if proof["traced_tactics"] != []:
        break
proof.keys()

dict_keys(['url', 'commit', 'file_path', 'full_name', 'start', 'end', 'traced_tactics'])

In [20]:
proof["url"], proof["commit"], proof["file_path"], proof["full_name"]

('https://github.com/leanprover-community/mathlib4',
 '355541ae7a2455222f179dcf7f074aa2c45eb8aa',
 'Mathlib/Topology/Algebra/InfiniteSum/Basic.lean',
 "tsum_comm'")

We see the theorem's name and where it is defined. The thereom includes some traced tactics.

In [21]:
len(proof["traced_tactics"])

2

Let's look at a traced tactic.

In [22]:
proof["traced_tactics"][0]

{'tactic': "erw [← tsum_prod' h h₁, ← tsum_prod' h.prod_symm h₂, ← (Equiv.prodComm γ β).tsum_eq (uncurry f)]",
 'annotated_tactic': ["erw [← tsum_prod' h h₁, ← tsum_prod' h.prod_symm h₂, ← (<a>Equiv.prodComm</a> γ β).tsum_eq (<a>uncurry</a> f)]",
  [{'full_name': 'Equiv.prodComm',
    'def_path': 'Mathlib/Logic/Equiv/Basic.lean',
    'def_pos': [114, 5],
    'def_end_pos': [114, 13]},
   {'full_name': 'Function.uncurry',
    'def_path': 'Mathlib/Init/Function.lean',
    'def_pos': [216, 5],
    'def_end_pos': [216, 12]}]],
 'state_before': "α : Type u_1\nβ : Type u_2\nγ : Type u_3\nδ : Type u_4\ninst✝⁷ : AddCommMonoid α\ninst✝⁶ : TopologicalSpace α\nf✝ g : β → α\na a₁ a₂ : α\ninst✝⁵ : T2Space α\ninst✝⁴ : ContinuousAdd α\ninst✝³ : AddCommMonoid δ\ninst✝² : TopologicalSpace δ\ninst✝¹ : T3Space δ\ninst✝ : ContinuousAdd δ\nf : β → γ → δ\nh : Summable (uncurry f)\nh₁ : ∀ (b : β), Summable (f b)\nh₂ : ∀ (c : γ), Summable fun b => f b c\n⊢ ∑' (c : γ) (b : β), f b c = ∑' (b : β) (c : γ), f b c

`annotated_tactic` is the tactic with premises annotated by `<a> ... </a>`. For each premise, we know its fully qualified name and the exact location it is defined, which is invaluable for training machine learning models for premise selection.